In [1]:
import yfinance as yf
import pandas as pd

symbols = {
    "BTC-USD": "BTC",
    "ETH-USD": "ETH",
    "SOL-USD": "SOL"
}

all_data = []

for yf_symbol, short_symbol in symbols.items():
    ticker = yf.Ticker(yf_symbol)
    hist = ticker.history(period="max")
    hist = hist.reset_index()
    hist["Symbol"] = short_symbol
    all_data.append(hist)

historical_df = pd.concat(all_data, ignore_index=True)
historical_df = historical_df[["Symbol", "Date", "Open", "High", "Low", "Close", "Volume"]]

historical_df.tail()
#historical_df[df["Symbol"] == "SOL"].tail()

,Symbol,Date,Open,High,Low,Close,Volume
8844,SOL,2025-09-04 00:00:00+00:00,210.746307,211.795013,202.127014,202.560181,5241887525
8845,SOL,2025-09-05 00:00:00+00:00,202.546738,209.860062,201.180008,203.498520,7780086538
8846,SOL,2025-09-06 00:00:00+00:00,203.498520,204.557541,199.640396,200.246307,3057210360
8847,SOL,2025-09-07 00:00:00+00:00,200.247360,207.867340,200.228500,206.468185,4153166545
8848,SOL,2025-09-08 00:00:00+00:00,206.444824,216.671661,205.853394,215.556335,9752061952


In [2]:
historical_df['Symbol'].unique()

array(['BTC', 'ETH', 'SOL'], dtype=object)

In [3]:
historical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8849 entries, 0 to 8848
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Symbol  8849 non-null   object             
 1   Date    8849 non-null   datetime64[ns, UTC]
 2   Open    8849 non-null   float64            
 3   High    8849 non-null   float64            
 4   Low     8849 non-null   float64            
 5   Close   8849 non-null   float64            
 6   Volume  8849 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 484.1+ KB


In [4]:
#historical_df.to_csv("historical_prices.csv", index=False)

In [5]:
def enforce_schema(df):
    schema = {
        "Symbol": "string",
        "Date": "datetime64[ns]",
        "Open": "float64",
        "High": "float64",
        "Low": "float64",
        "Close": "float64",   # nullable int
        "Volume": "Int64",
    }

    # apply schema selectively (ignore cols not in dict)
    for col, dtype in schema.items():
        if col in df.columns:
            if dtype.startswith("datetime"):
                df[col] = pd.to_datetime(df[col], errors="coerce")
            else:
                df[col] = df[col].astype(dtype, errors="ignore")
    return df


In [6]:
silver_historical_df = enforce_schema(historical_df)
silver_historical_df.head()

,Symbol,Date,Open,High,Low,Close,Volume
0,BTC,2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800
1,BTC,2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200
2,BTC,2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700
3,BTC,2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600
4,BTC,2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100


In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
#MONGO_URI = os.getenv("MONGO_URI")

True

In [8]:
import os
import psycopg2 #for connecting to PostgreSQL database and executing queries
from sqlalchemy import create_engine #To efficiently manage and reuse datavase connections
load_dotenv()

DB_USERNAME = os.getenv('DB_USERNAME', 'postgres')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'ben')

In [9]:
from sqlalchemy import create_engine, text
import pandas as pd
# Create engine
engine = create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [10]:
 try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        print("Connected to:", result.scalar())
except Exception as e:
    print("Connection failed:", e)


Connected to: PostgreSQL 17.5 on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit


In [21]:
with engine.begin() as conn:
    conn.execute(text("""
         DROP TABLE IF EXISTS yfinance_historical;

        CREATE TABLE yfinance_historical (
            Symbol VARCHAR(10),
            Date TIMESTAMP,
            Open FLOAT,
            High FLOAT,
            Low FLOAT,
            Close FLOAT,
            Volume BIGINT
        );
    """))


In [22]:
from sqlalchemy import inspect

inspector = inspect(engine)
print(inspector.get_table_names())


['prices', 'yfinance_historical', 'yfinance_hourly']


In [23]:
columns_to_keep = ["Symbol","Date", "Open", "High","Low","Close","Volume"]

# Keep only the desired columns
df_subset = silver_historical_df[columns_to_keep]

# Optionally lowercase column names (useful for Postgres)
df_subset.columns = [col.lower() for col in df_subset.columns]

# Push into SQL
df_subset.to_sql(
    "yfinance_historical",
    con=engine,
    if_exists="append",
    index=False
)


849